# Functions to Update Alert Database

# Prep

## Import Packages

In [1]:
# File Manipulation

import os # For working with Operating System
import sys # System arguments
from dotenv import load_dotenv # Loading .env info

# Web

import requests # Accessing the Web

# Time

import datetime as dt # Working with dates/times
import pytz # Timezones

# Database 

import psycopg2
from psycopg2 import sql

# Data Manipulation

import numpy as np
import geopandas as gpd
import pandas as pd

### Load Functions

In [2]:
script_path = os.path.join('..', '..', 'Scripts', 'python')

# Function definition - Please see Scripts/python/*
exec(open(os.path.join(script_path, 'Get_spikes_df.py')).read())

### Global Variables

In [3]:
load_dotenv() # Load .env file

## API Keys

purpleAir_api = os.getenv('PURPLEAIR_API_TOKEN') # PurpleAir API Read Key

## Database credentials

creds = [os.getenv('DB_NAME'),
         os.getenv('DB_USER'),
         os.getenv('DB_PASS'),
         os.getenv('DB_PORT'),
         os.getenv('DB_HOST')
        ]

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))  

# Other Constants from System Arguments

spike_threshold = 35 # Value which defines an AQ_Spike (Micgrograms per meter cubed)

timestep = 10 # Sleep time in between updates (in Minutes)

# When to stop the program? (datetime)
days_to_run = 7 # How many days will we run this?
stoptime = dt.datetime.now() + dt.timedelta(days=days_to_run)

## Compute/Define other necessary variables 

In [4]:
#  Get the sensor_ids from sensors in our database

sensor_ids = get_sensor_ids(pg_connection_dict) # In Get_Spikes_df.py

In [5]:
# Get Spikes Dataframe and runtime

spikes_df, runtime = Get_spikes_df(purpleAir_api, sensor_ids, spike_threshold) # In Get_Spikes_df.py

# Definitions

## Initial Functions

### Get Active Alerts

In [6]:
# Get active alerts from database

conn = psycopg2.connect(**pg_connection_dict)

# Create json cursor
cur = conn.cursor()

cmd = sql.SQL('''SELECT * 
FROM "Active Alerts Acute PurpleAir"
''')

cur.execute(cmd) # Execute

conn.commit() # Committ command

# Convert response into dataframe

cols_for_active_alerts = ['alert_index', 'sensor_indices', 'start_time', 'max_reading']
active_alerts = pd.DataFrame(cur.fetchall(), columns = cols_for_active_alerts)

# Close cursor
cur.close()
# Close connection
conn.close()

In [7]:
active_alerts

,alert_index,sensor_indices,start_time,max_reading
0,16,[142752],2023-10-27 15:20:59,27.0
1,3,[143944],2023-10-26 02:58:16,79.0
2,17,[145202],2023-10-27 15:20:59,28.3
3,18,[145504],2023-10-27 15:20:59,25.6
4,19,[145610],2023-10-27 15:20:59,32.8


### Sort the Sensor IDs

In [8]:
# Check for 4 types of Sensor ID
# Using set operations between:

# Currently active
current_active_spike_sensors = set(spikes_df.sensor_index) # From most recent api call

# Previously active
if len(active_alerts) > 0:
    previous_active_spike_sensors = set(active_alerts.sensor_indices.sum()) # From our database
    # The sensor_indices are given as lists of indices because we may cluster alerts eventually
else:
    previous_active_spike_sensors = set()

# The sets:

# 1) new
new_spike_sensors = current_active_spike_sensors - previous_active_spike_sensors

# 2) ongoing, 
ongoing_spike_sensors = current_active_spike_sensors.intersection(previous_active_spike_sensors)

# 3) ended alerts
ended_spike_sensors = previous_active_spike_sensors - current_active_spike_sensors

# 4) Not Spiked
not_spiked_sensors = set(sensor_ids.astype(int)) - current_active_spike_sensors

In [9]:
new_spike_sensors

set()

In [10]:
ongoing_spike_sensors

{143944}

In [11]:
ended_spike_sensors

{142752, 145202, 145504, 145610}

In [12]:
# not_spiked_sensors

## New Alerts

For each new alert, we should:

In [12]:
# Initialize for For Loop

# Iterable (Find rows in spikes_df that are new spikes)

new_spikes = spikes_df[spikes_df.sensor_index.isin(new_spike_sensors)]

### Add to Active Alerts

In [14]:
# 1) Add to active alerts

def add_to_active_alerts(row, pg_connection_dict, cols_for_db, runtime_for_db):

    sensor_index = row.sensor_index
    reading = row.pm25

    # 1) Add to active alerts

    # Create Cursor for commands
    conn = psycopg2.connect(**pg_connection_dict)
    cur = conn.cursor()
    
    # This is really a great way to insert a lot of data

    vals = [[sensor_index], runtime_for_db, reading]
    
    q1 = sql.SQL('INSERT INTO "Active Alerts Acute PurpleAir" ({}) VALUES ({});').format(
     sql.SQL(', ').join(map(sql.Identifier, cols_for_db)),
     sql.SQL(', ').join(sql.Placeholder() * (len(cols_for_db))))

    cur.execute(q1.as_string(conn),
        (vals)
        )
    # Commit command
    conn.commit()

    # Get the alert_index that was just created
    
    cmd = sql.SQL('''SELECT alert_index
FROM "Active Alerts Acute PurpleAir"
WHERE sensor_indices = {}::int[];'''
             ).format(sql.Literal([sensor_index]))
    
    cur.execute(cmd)     
    
    conn.commit() # Committ command
    
    newest_alert_index = cur.fetchall()[0][0]

    # Close cursor
    cur.close()
    # Close connection
    conn.close()

    return newest_alert_index

### Update User's Active Alerts - NOT DONE

Add these alerts to user's active_alerts in database
                    
Potential Query: Select record_ids 
                 From "Sign Up Information"
                 Where sensors of interest intersect with the an alert's sensor_indices

### For loop for Experimenting

In [15]:
# The For Loop

if len(new_spike_sensors) > 0: # If there are new spikes
    
    # Variables for Database

    cols_for_db = ['sensor_index', 'start_time', 'max_reading'] # Cols_for_db What are we keeping track of in active alerts?
    
    runtime_for_db = runtime.strftime('%Y-%m-%d %H:%M:%S') # The time PurpleAir API queried

    # Iterate through new spikes
    
    for index, row in new_spikes.iterrows():

        # 1) Add to active alerts
    
        add_to_active_alerts(row, pg_connection_dict, cols_for_db, runtime_for_db)

## Ongoing Alerts

In [16]:
ongoing_spike_sensors

{143944}

In [17]:
# Initialize for For Loop

# Iterable (Find rows in spikes_df that are new spikes)

ongoing_spikes = spikes_df[spikes_df.sensor_index.isin(ongoing_spike_sensors)]

ongoing_spikes

,sensor_index,pm25
35,143944,79.0


In [18]:
# Function Definitions

# For each ongoing alert, we should

# 1) Update max_reading if it's higher

def update_max_reading(row, pg_connection_dict):
    '''
    Row should be a row from the ongoing_spikes dataFrame
    '''

    sensor_index = row.sensor_index
    reading = row.pm25

    # 1) Add to active alerts

    # Create Cursor for commands
    conn = psycopg2.connect(**pg_connection_dict)
    cur = conn.cursor()
    
    cmd = sql.SQL('''
UPDATE "Active Alerts Acute PurpleAir"
SET max_reading = GREATEST({}, max_reading)
WHERE {} = ANY (sensor_indices);

''').format(sql.Literal(reading), sql.Literal(sensor_index))

    cur.execute(cmd)
    # Commit command
    conn.commit()

    # Close cursor
    cur.close()
    # Close connection
    conn.close()

In [19]:
# The For Loop

if len(ongoing_spikes) > 0:

    for _, spike in ongoing_spikes.iterrows():
    
        update_max_reading(spike, pg_connection_dict)

## Ended Alerts

In [ ]:
# # # To manufacture an ended alert

# new_spike_sensors = new_spike_sensors - {143944}
# ended_spike_sensors = {143944}

In [ ]:
# print(not_spiked_sensors)

In [ ]:
# Function Definitions

# For each ended alert, we should

# 1) Add to archived alerts

def add_to_archived_alerts(not_spiked_sensors, pg_connection_dict):
    '''
    '''

    # Get relevant sensor indices as list
    sensor_indices = list(not_spiked_sensors)
                           
    # Create Cursor for commands
    conn = psycopg2.connect(**pg_connection_dict)
    cur = conn.cursor()

    # This command selects the ended alerts from active alerts
    # Then it gets the difference from the current time and when it started
    # Lastly, it inserts this selection while converting that time difference into minutes for duration_minutes column
    cmd = sql.SQL('''
    WITH ended_alerts as
    (
    	SELECT alert_index, sensor_indices, start_time, CURRENT_TIMESTAMP AT TIME ZONE 'America/Chicago' - start_time as time_diff, max_reading 
     	FROM "Active Alerts Acute PurpleAir"
     	WHERE sensor_indices <@ {}::int[] -- contained
    )
    INSERT INTO "Archived Alerts Acute PurpleAir" 
    SELECT alert_index, sensor_indices, start_time, (((DATE_PART('day', time_diff) * 24) + 
    	DATE_PART('hour', time_diff)) * 60 + DATE_PART('minute', time_diff)) as duration_minutes, max_reading
    FROM ended_alerts;
    ''').format(sql.Literal(sensor_indices))
    
    cur.execute(cmd)
    # Commit command
    conn.commit()
    
    # Close cursor
    cur.close()
    # Close connection
    conn.close()
    

#~~~~~~~~~~~~~~~~

In [ ]:
# 2) Remove from active alerts

def remove_active_alerts(not_spiked_sensors, pg_connection_dict):
    '''
    This function removes the ended_spikes from the Active Alerts Table
    It also retrieves their alert_index
    
    ended_spike_sensors is a set of sensor indices that have ended spikes Alerts
    
    ended_alert_indices is returned alert_index of the removed alerts for accessing Archive for end message 
    
    '''

    # Get relevant sensor indices as list
    sensor_indices = list(not_spiked_sensors)
                           
    # Create Cursor for commands
    conn = psycopg2.connect(**pg_connection_dict)
    cur = conn.cursor()
    
    cmd = sql.SQL('''
    SELECT alert_index
    FROM "Active Alerts Acute PurpleAir"
    WHERE sensor_indices <@ {}::int[]; -- contained
    ''').format(sql.Literal(sensor_indices))
    
    cur.execute(cmd)
    # Commit command
    conn.commit()
    
    ended_alert_indices = [i[0] for i in cur.fetchall()]
    
    cmd = sql.SQL('''
    DELETE FROM "Active Alerts Acute PurpleAir"
    WHERE sensor_indices <@ {}::int[]; -- contained
    ''').format(sql.Literal(sensor_indices))
    
    cur.execute(cmd)
    # Commit command
    conn.commit()
    
    # Close cursor
    cur.close()
    # Close connection
    conn.close()

    return ended_alert_indices

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~    
  

In [ ]:
# To test

if len(ended_spike_sensors) > 0:

    add_to_archived_alerts(not_spiked_sensors.union(ended_spike_sensors), pg_connection_dict) # Add the ended SpikeAlerts to archive
    
    ended_alert_indices = remove_active_alerts(not_spiked_sensors.union(ended_spike_sensors), pg_connection_dict) # Remove them from Active Alerts

    print(ended_alert_indices)

In [ ]:
  
# 3) Transfer these alerts from "Sign Up Information" active_alerts to "Sign Up Information" cached_alerts

def cache_alerts(ended_alert_indices, pg_connection_dict):
    '''
    NOT DONE
    '''

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~    
   

In [ ]:
 
# 4-5b) Query for people to text, initialize reports, text people - see Send_Alerts.py

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~    

In [ ]:
# 5c) Clear a user's cache

def clear_cached_alerts(record_id, pg_connection_dict):
    '''
    NOT DONE
    '''